<a href="https://colab.research.google.com/github/zyadalazazi/Data-Glacier-Work/blob/main/File-Ingestion-and-Schema-Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# File Ingestion and Schema Validation

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [3]:
cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d solarmainframe/ids-intrusion-csv

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip \*.zip && rm *.zip

### Importing the Dataset

In [1]:
import pandas as pd
import time

In [2]:
start_time = time.time()

panda_df = pd.read_csv("/content/gdrive/My Drive/Kaggle/02-20-2018.csv", dtype={'Active Mean': 'double', 'Active Std': 'double', 'Bwd IAT Max': 'double'}, delimiter = ',')

end_time = time.time()
execution_time = end_time - start_time
print('Execution time using Pandas in seconds: '+ str(execution_time))

Execution time using Pandas in seconds: 107.0230028629303


In [17]:
!pip install dask[dataframe]

     |████████████████████████████████| 112kB 3.8MB/s 


In [3]:
import dask.dataframe as dd

start_time2 = time.time()

dask_df = dd.read_csv("/content/gdrive/My Drive/Kaggle/02-20-2018.csv")

end_time2 = time.time()
execution_time2 = end_time2 - start_time2
print('Execution time using Dask in seconds: '+ str(execution_time2))

Execution time using Dask in seconds: 0.3253915309906006


We can see that using dask has resuted in a very significant speedup. Thus, we will be implementing our operation on the dask dataframe since our dataset has a size of over 3 GB.

In [4]:
dask_df.columns

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE 

In [5]:
dask_df.columns = dask_df.columns.str.replace(' ', "_")

In [6]:
dask_df.columns

Index(['Flow_ID', 'Src_IP', 'Src_Port', 'Dst_IP', 'Dst_Port', 'Protocol',
       'Timestamp', 'Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts',
       'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max',
       'Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std',
       'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Std', 'Flow_Byts/s', 'Flow_Pkts/s', 'Flow_IAT_Mean',
       'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot',
       'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min',
       'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max',
       'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags',
       'Bwd_URG_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len', 'Fwd_Pkts/s',
       'Bwd_Pkts/s', 'Pkt_Len_Min', 'Pkt_Len_Max', 'Pkt_Len_Mean',
       'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt', 'SYN_Flag_Cnt',
       'RST_Flag_Cnt', 'PSH_Flag_Cnt', 'ACK_Flag_Cnt', 'URG_Flag_Cnt',
       'CWE_Flag_Count', 'ECE_

In [7]:
dask_df

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,Bwd_Pkt_Len_Max,Bwd_Pkt_Len_Min,Bwd_Pkt_Len_Mean,Bwd_Pkt_Len_Std,Flow_Byts/s,Flow_Pkts/s,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Tot,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Tot,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Len,Bwd_Header_Len,Fwd_Pkts/s,Bwd_Pkts/s,Pkt_Len_Min,Pkt_Len_Max,Pkt_Len_Mean,Pkt_Len_Std,Pkt_Len_Var,FIN_Flag_Cnt,SYN_Flag_Cnt,RST_Flag_Cnt,PSH_Flag_Cnt,ACK_Flag_Cnt,URG_Flag_Cnt,CWE_Flag_Count,ECE_Flag_Cnt,Down/Up_Ratio,Pkt_Size_Avg,Fwd_Seg_Size_Avg,Bwd_Seg_Size_Avg,Fwd_Byts/b_Avg,Fwd_Pkts/b_Avg,Fwd_Blk_Rate_Avg,Bwd_Byts/b_Avg,Bwd_Pkts/b_Avg,Bwd_Blk_Rate_Avg,Subflow_Fwd_Pkts,Subflow_Fwd_Byts,Subflow_Bwd_Pkts,Subflow_Bwd_Byts,Init_Fwd_Win_Byts,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
npartitions=64,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,int64,object,int64,int64,object,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,int64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Creating the YAML file

In [26]:
!pip install schema

In [8]:
%%writefile testutility.py
import logging
import subprocess
import datetime
import gc
import re
from schema import Schema, SchemaError
import yaml
import os

def read_yml_file(file_path):
    with open(file_path, 'r') as stream:
      try:
          return yaml.safe_load(stream)
      except yaml.YAMLError as exec:
          logging.error(exec)

def substitutor(string, char):
  pattern = char + '{2,}'
  string = re.sub(pattern, char, string)
  return string

def replacer(string, char):
  pattern = char + '{2,}'
  string = re.sub(pattern, char, string) 
  return string

def col_validate(df, col_config):
  colms = df.columns
  colms = colms.str.replace(' ','_')
  #colms = list(map(lambda x: x.strip('_'), list(colms)))
  colms = list(map(lambda x: replacer(x,'_'), list(colms)))
  expected_col = list(map(lambda x: x.lower(),  col_config['columns']))
  colms = list(map(lambda x: x.lower(), list(colms)))

  print("real: ", colms)
  print("exp: ", expected_col)

  if len(colms) == len(expected_col) and list(colms) == list(expected_col):
    print("\nColumn name and column length are successfully validated!")
    return 1

  else:
    print("Column name and column length have not passed the validation test")
    mismatched_columns = list(set(colms).difference(expected_col))
    print("The following columns are missing from the YAML file", mismatched_columns)
    missing_file = list(set(expected_col).difference(colms))
    print("The followning columns are missing from the file uploaded", missing_file)
    logging.info(f'df columns: {colms}')
    logging.info(f'expected columns: {expected_col}')
    return 0
    
def stats(df, config_data):
  col_number = df.shape[1]
  row_number = df.shape[0] 
  print("\nStatistics About the Dataset:")
  print("Number of columns is: ", col_number)
  print("Number of rows is: ", row_number)
  size = os.path.getsize('/content/gdrive/My Drive/Kaggle/02-20-2018.csv')
  GB_size = size/(1024*1024*1024)
  print("Size of the file is: ", round(GB_size, 3), "GB")

def save2gz(df):
  df.to_csv("dataset.gz", sep = '|', compression = 'gzip')
  success = "Dataset was compressed and saved in compressed_dataset.gz"
  return success

Writing testutility.py


In [9]:
%%writefile dataset.yaml
file_type: csv
dataset_name: intrusions
file_name: 02-20-2018
table_name: 20th_of_April
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
 - flow_id
 - src_ip
 - src_port
 - dst_ip
 - dst_port
 - protocol
 - timestamp
 - flow_duration
 - tot_fwd_pkts
 - tot_bwd_pkts
 - totlen_fwd_pkts
 - totlen_bwd_pkts
 - fwd_pkt_len_max
 - fwd_pkt_len_min
 - fwd_pkt_len_mean
 - fwd_pkt_len_std
 - bwd_pkt_len_max
 - bwd_pkt_len_min
 - bwd_pkt_len_mean
 - bwd_pkt_len_std
 - flow_byts/s
 - flow_pkts/s
 - flow_iat_mean
 - flow_iat_std
 - flow_iat_max
 - flow_iat_min
 - fwd_iat_tot
 - fwd_iat_mean
 - fwd_iat_std
 - fwd_iat_max
 - fwd_iat_min
 - bwd_iat_tot
 - bwd_iat_mean
 - bwd_iat_std
 - bwd_iat_max
 - bwd_iat_min
 - fwd_psh_flags
 - bwd_psh_flags
 - fwd_urg_flags
 - bwd_urg_flags
 - fwd_header_len
 - bwd_header_len
 - fwd_pkts/s
 - bwd_pkts/s
 - pkt_len_min
 - pkt_len_max
 - pkt_len_mean
 - pkt_len_std
 - pkt_len_var
 - fin_flag_cnt
 - syn_flag_cnt
 - rst_flag_cnt
 - psh_flag_cnt
 - ack_flag_cnt
 - urg_flag_cnt
 - cwe_flag_count
 - ece_flag_cnt
 - down/up_ratio
 - pkt_size_avg
 - fwd_seg_size_avg
 - bwd_seg_size_avg
 - fwd_byts/b_avg
 - fwd_pkts/b_avg
 - fwd_blk_rate_avg
 - bwd_byts/b_avg
 - bwd_pkts/b_avg
 - bwd_blk_rate_avg
 - subflow_fwd_pkts
 - subflow_fwd_byts
 - subflow_bwd_pkts
 - subflow_bwd_byts
 - init_fwd_win_byts
 - init_bwd_win_byts
 - fwd_act_data_pkts
 - fwd_seg_size_min
 - active_mean
 - active_std
 - active_max
 - active_min
 - idle_mean
 - idle_std
 - idle_max
 - idle_min
 - label

Writing dataset.yaml


### Reading the Config File

In [10]:
 import testutility as util
 config_data = util.read_yml_file("dataset.yaml")

In [11]:
config_data

{'columns': ['flow_id',
  'src_ip',
  'src_port',
  'dst_ip',
  'dst_port',
  'protocol',
  'timestamp',
  'flow_duration',
  'tot_fwd_pkts',
  'tot_bwd_pkts',
  'totlen_fwd_pkts',
  'totlen_bwd_pkts',
  'fwd_pkt_len_max',
  'fwd_pkt_len_min',
  'fwd_pkt_len_mean',
  'fwd_pkt_len_std',
  'bwd_pkt_len_max',
  'bwd_pkt_len_min',
  'bwd_pkt_len_mean',
  'bwd_pkt_len_std',
  'flow_byts/s',
  'flow_pkts/s',
  'flow_iat_mean',
  'flow_iat_std',
  'flow_iat_max',
  'flow_iat_min',
  'fwd_iat_tot',
  'fwd_iat_mean',
  'fwd_iat_std',
  'fwd_iat_max',
  'fwd_iat_min',
  'bwd_iat_tot',
  'bwd_iat_mean',
  'bwd_iat_std',
  'bwd_iat_max',
  'bwd_iat_min',
  'fwd_psh_flags',
  'bwd_psh_flags',
  'fwd_urg_flags',
  'bwd_urg_flags',
  'fwd_header_len',
  'bwd_header_len',
  'fwd_pkts/s',
  'bwd_pkts/s',
  'pkt_len_min',
  'pkt_len_max',
  'pkt_len_mean',
  'pkt_len_std',
  'pkt_len_var',
  'fin_flag_cnt',
  'syn_flag_cnt',
  'rst_flag_cnt',
  'psh_flag_cnt',
  'ack_flag_cnt',
  'urg_flag_cnt',
  'cwe_

In [12]:
src_file = "./"+config_data['file_name']+"."+config_data['file_type']
#print(src_file)
df_dask = dd.read_csv("/content/gdrive/My Drive/Kaggle/"+src_file)
df_dask.head

<bound method _Frame.head of Dask DataFrame Structure:
               Flow ID  Src IP Src Port  Dst IP Dst Port Protocol Timestamp Flow Duration Tot Fwd Pkts Tot Bwd Pkts TotLen Fwd Pkts TotLen Bwd Pkts Fwd Pkt Len Max Fwd Pkt Len Min Fwd Pkt Len Mean Fwd Pkt Len Std Bwd Pkt Len Max Bwd Pkt Len Min Bwd Pkt Len Mean Bwd Pkt Len Std Flow Byts/s Flow Pkts/s Flow IAT Mean Flow IAT Std Flow IAT Max Flow IAT Min Fwd IAT Tot Fwd IAT Mean Fwd IAT Std Fwd IAT Max Fwd IAT Min Bwd IAT Tot Bwd IAT Mean Bwd IAT Std Bwd IAT Max Bwd IAT Min Fwd PSH Flags Bwd PSH Flags Fwd URG Flags Bwd URG Flags Fwd Header Len Bwd Header Len Fwd Pkts/s Bwd Pkts/s Pkt Len Min Pkt Len Max Pkt Len Mean Pkt Len Std Pkt Len Var FIN Flag Cnt SYN Flag Cnt RST Flag Cnt PSH Flag Cnt ACK Flag Cnt URG Flag Cnt CWE Flag Count ECE Flag Cnt Down/Up Ratio Pkt Size Avg Fwd Seg Size Avg Bwd Seg Size Avg Fwd Byts/b Avg Fwd Pkts/b Avg Fwd Blk Rate Avg Bwd Byts/b Avg Bwd Pkts/b Avg Bwd Blk Rate Avg Subflow Fwd Pkts Subflow Fwd Byts Subf

### Checking the Validity and Writing to .gz File

In [14]:
if util.col_validate(panda_df, config_data)==0:
  print("Validation failed")
else:
  print("\nValidation completed successfully!")
  util.stats(panda_df, config_data)

real:  ['flow_id', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'protocol', 'timestamp', 'flow_duration', 'tot_fwd_pkts', 'tot_bwd_pkts', 'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max', 'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std', 'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean', 'bwd_pkt_len_std', 'flow_byts/s', 'flow_pkts/s', 'flow_iat_mean', 'flow_iat_std', 'flow_iat_max', 'flow_iat_min', 'fwd_iat_tot', 'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max', 'fwd_iat_min', 'bwd_iat_tot', 'bwd_iat_mean', 'bwd_iat_std', 'bwd_iat_max', 'bwd_iat_min', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags', 'bwd_urg_flags', 'fwd_header_len', 'bwd_header_len', 'fwd_pkts/s', 'bwd_pkts/s', 'pkt_len_min', 'pkt_len_max', 'pkt_len_mean', 'pkt_len_std', 'pkt_len_var', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt', 'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'cwe_flag_count', 'ece_flag_cnt', 'down/up_ratio', 'pkt_size_avg', 'fwd_seg_size_avg', 'bwd_seg_size_avg', 'fwd_byts/b_a

In [15]:
util.save2gz(panda_df)

'Dataset was compressed and saved in compressed_dataset.gz'